In [1]:
# This ipynb was created in late 2020 after referencing MLPlus and NLP with Spacy. The aim was to create a NER trianing for compensation consultnat.
# This file has been superceded by train_CC.py & test_CC.py in Jan, 2021.
%pwd

'C:\\Users\\KWan\\Documents\\NLP & ML'

In [2]:
import spacy
nlp=spacy.load('en_core_web_lg')

In [3]:

nlp = spacy.load('en_core_web_sm')
doc = nlp(u"The firm earned $1.5 million in 2017. This is the next sentence.")
phrase = ''

for i, sent in enumerate(doc.sents):
    print(f'{i}: {sent}')

0: The firm earned $1.5 million in 2017.
1: This is the next sentence.


In [3]:
CC=[
    'Deloitte','Aon','Aon Hewitt','Compensation Advisory Partners','Mercer HR Consulting',
   'Hewitt','Compensia','Exequity','FPL Advisory Group Co.','Frederic W. Cook & Co., Inc.', 'Frederic W. Cook & Co.','F.W. Cook','F. W. Cook', 'Cook & Co.',
    'FTI Consulting Inc.','Hay Group', 'The Hay Group','Hewitt Associates','Longecker & Associates',
    'Mercer','Mercer Human Resources Consulting','Mercer Human Resource Consulting','Meridian','Meridian Compensation',
    'Pay Governance','Pearl Meyer & Partners','Radford','Semler Brossy Consulting',
    'Semler Brossy','Steven Hall & Partners','Towers Watson','Watson Wyatt','Towers Perrin','Watson Wyatt Worldwide',
    'CCA Strategies','Total Rewards Strategies'
]

In [4]:
from spacy.pipeline import EntityRuler
# Initialize
ruler = EntityRuler(nlp)

patterns=[]
for item in CC:
    pattern={"label": "CC", "pattern": item}
    patterns.append(pattern)
patterns

[{'label': 'CC', 'pattern': 'Deloitte'},
 {'label': 'CC', 'pattern': 'Aon'},
 {'label': 'CC', 'pattern': 'Aon Hewitt'},
 {'label': 'CC', 'pattern': 'Compensation Advisory Partners'},
 {'label': 'CC', 'pattern': 'Mercer HR Consulting'},
 {'label': 'CC', 'pattern': 'Hewitt'},
 {'label': 'CC', 'pattern': 'Compensia'},
 {'label': 'CC', 'pattern': 'Exequity'},
 {'label': 'CC', 'pattern': 'FPL Advisory Group Co.'},
 {'label': 'CC', 'pattern': 'Frederic W. Cook & Co., Inc.'},
 {'label': 'CC', 'pattern': 'Frederic W. Cook & Co.'},
 {'label': 'CC', 'pattern': 'F.W. Cook'},
 {'label': 'CC', 'pattern': 'F. W. Cook'},
 {'label': 'CC', 'pattern': 'Cook & Co.'},
 {'label': 'CC', 'pattern': 'FTI Consulting Inc.'},
 {'label': 'CC', 'pattern': 'Hay Group'},
 {'label': 'CC', 'pattern': 'The Hay Group'},
 {'label': 'CC', 'pattern': 'Hewitt Associates'},
 {'label': 'CC', 'pattern': 'Longecker & Associates'},
 {'label': 'CC', 'pattern': 'Mercer'},
 {'label': 'CC', 'pattern': 'Mercer Human Resources Consult

In [5]:
ruler.add_patterns(patterns)
# Add entity ruler to the NLP pipeline. 
# NLP pipeline is a sequence of NLP tasks that spaCy performs for a given text
# More on pipelines coming in future section in this post.
nlp.add_pipe(ruler)

In [6]:
# this does not mean token will be done automatically
# token may not be an issue, though in real applications.

In [7]:
doc=nlp('Frederic W. Cook & Co.')
for token in doc:
    print(token.text, token.ent_type_)
# tokenization makes no difference

Frederic ORG
W. ORG
Cook ORG
& ORG
Co. ORG


In [8]:
# retokenizer
doc = nlp("Frederic W. Cook & Co.")
with doc.retokenize() as retokenizer:
    retokenizer.merge(doc[0:])
for token in doc:
    print(token)

Frederic W. Cook & Co.


In [9]:
# retokenization further
## It would be better if ent_label_ can be re-done as well.
for item in CC:
    doc=nlp(item)
    with doc.retokenize() as retokenizer:
        retokenizer.merge(doc[0:])
    for token in doc:
        print(token)

Deloitte
Aon
Aon Hewitt
Compensation Advisory Partners
Mercer HR Consulting
Hewitt
Compensia
Exequity
FPL Advisory Group Co.
Frederic W. Cook & Co., Inc.
Frederic W. Cook & Co.
F.W. Cook
F. W. Cook
Cook & Co.
FTI Consulting Inc.
Hay Group
The Hay Group
Hewitt Associates
Longecker & Associates
Mercer
Mercer Human Resources Consulting
Mercer Human Resource Consulting
Meridian
Meridian Compensation
Pay Governance
Pearl Meyer & Partners
Radford
Semler Brossy Consulting
Semler Brossy
Steven Hall & Partners
Towers Watson
Watson Wyatt
Towers Perrin
Watson Wyatt Worldwide
CCA Strategies
Total Rewards Strategies


In [10]:
# Save the  model to directory
output_dir = 'C:\\Users\\KWan\\Desktop'
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
# created entity_ruler folder

Saved model to C:\Users\KWan\Desktop


In [11]:
# Load the saved model and predict
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)


Loading from C:\Users\KWan\Desktop


C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\spacy\_ml.py:304: UserWarning: [W019] Changing vectors name from en_core_web_lg.vectors to en_core_web_lg.vectors_890280, to avoid clash with previously loaded vectors. See Issue #3853.
  warnings.warn(Warnings.W019.format(old=old_name, new=new_name))


In [12]:
# test
doc = nlp_updated('''
    Frederic W. Cook & Co. and Compensation Advisory Partners can be ordered in FlipKart. Mercer HR Consulting is Mercer. Semler Brossy is another company. 
    Mercer HR Consulting is not Steven Hall & Partners.
    Lastly, Meridian is not Aon Hewitt nor Hewitt Associates. Hewitt is Aon Hewitt
    ''')

print("Entities", [(ent.text, ent.label_) for ent in doc.ents])## test

Entities [('Frederic W. Cook & Co.', 'ORG'), ('Compensation Advisory Partners', 'ORG'), ('FlipKart', 'ORG'), ('Mercer HR Consulting', 'ORG'), ('Mercer', 'GPE'), ('Semler Brossy', 'PERSON'), ('Mercer HR Consulting', 'ORG'), ('Steven Hall & Partners', 'ORG'), ('Meridian', 'ORG'), ('Aon Hewitt', 'PERSON'), ('Hewitt Associates', 'ORG'), ('Hewitt', 'PERSON'), ('Aon Hewitt', 'PERSON')]


In [ ]:
# To-do
# 1. need to make train data like below
# 2. to make this train data, you can look at the book.


In [13]:
# open the csv file for training
import csv
import re

ccFile=open('compensationConsultantTraining2.csv', encoding='utf-8-sig')
fileReader=csv.reader(ccFile)
string=''
for line in fileReader:
    line=str(line[0])
    line=re.sub(r'\xa0', '', line)
    line=re.sub(r'\n', '', line)
    string=string + line
#ccFile.close()
string

'F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.The committee has engaged Hewitt Associates to provide counsel and advice on executive and non-employee director compensation matters. Hewitt, and its principal, report directly to the chair of the committee. The principal meets regularly, and as needed, with the committee in executive sessions, has direct access to the chair during and between meetings, and performs no other services for Abbott or its senior executives.The committee determines what variables it will instruct Hewitt to consider, and they include: peer groups against which performance and pay should be examined, financial metrics to be used to assess Abbott\'s relative performance, competitive long-term incentive practices in the market place, and compensation levels relative to market practice. The committee negotiates and approves any fees paid to Hewitt for t

In [14]:
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc=nlp_updated(string)
for ent in doc.ents:
    #if ent.label_=='ORG' or ent.label_=='PERSON' or ent.label_=='CC':
    print(ent.text, ent.label_)

Loading from C:\Users\KWan\Desktop
F.W. Cook PERSON
Company ORG
Committee ORG
Hewitt Associates ORG
Hewitt PERSON
Abbott ORG
Hewitt PERSON
Abbott PERSON
Hewitt PERSON
2008 DATE
approximately $239,109 MONEY
Hewitt ORG
Abbott PERSON
approximately $5.3 million MONEY
Aon Hewitt PERSON
Committee ORG
Aon Hewitt PERSON
2013 DATE
2014 DATE
the Compensation Committee ORG
Compensation Committee ORG
Meridian ORG
Abbott ORG
Abbott ORG
Aon Hewitt ORG
approximately $5.8 million MONEY
The Committee ORG
Meridian ORG
Committee ORG
the Chair of the Committee ORG
Committee ORG
Chair ORG
Abbott ORG
Committee ORG
Abbott ORG
Committee ORG
Meridian ORG
the New York Stock Exchange ORG
Meridian ORG
Committee ORG
Meridian ORG
Committee ORG
Meridian ORG
2015 DATE
Meridian ORG
Abbott ORG
Committee ORG
Mercer Human Resource Consulting ORG
Mercer PERSON
Mercer PERSON
Committee ORG
2004 DATE
2006 DATE
Mercer PERSON
Board ORG
the CompensationIn 2009 EVENT
the Compensation Committee ORG
Semler Brossy PERSON
Semler Bro

In [15]:
line_num=0
for sent in doc.sents:
    print("#: ", line_num, " ",sent)
    line_num+=1

#:  0   F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.
#:  1   The committee has engaged Hewitt Associates to provide counsel and advice on executive and non-employee director compensation matters.
#:  2   Hewitt, and its principal, report directly to the chair of the committee.
#:  3   The principal meets regularly, and as needed, with the committee in executive sessions, has direct access to the chair during and between meetings, and performs no other services for Abbott or its senior executives.
#:  4   The committee determines what variables it will instruct Hewitt to consider, and they include: peer groups against which performance and pay should be examined, financial metrics to be used to assess Abbott's relative performance, competitive long-term incentive practices in the market place, and compensation levels relative to market practice.
#:  5   The committee negot

# FW Cook retokenizing

for token in doc:
    print (token)

with doc.retokenize() as retokenizer:
    retokenizer.merge(doc[0:6])
for token in doc:
    print(token)

# tokenizing

In [ ]:
nlp_updated = spacy.load(output_dir)
doc=nlp_updated(string)
for token in doc:
    if token.ent_type_=="PERSON" or token.ent_type_=="CC" or token.ent_type_=="ORG":
        print(token)
# This shows ent and tokens are differently parsed.

In [17]:
# check noun chunks
line_num=0
for sent in doc.sents:
    print("#: ", line_num, " ",sent)
    line_num+=1
    for nouns in sent.noun_chunks:
        print(nouns)
        print()

#:  0   F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.
F.W. Cook

any other services

the Company

no compensation

respect

the services

the Committee

#:  1   The committee has engaged Hewitt Associates to provide counsel and advice on executive and non-employee director compensation matters.
The committee

Hewitt Associates

counsel

advice

executive and non-employee director compensation matters

#:  2   Hewitt, and its principal, report directly to the chair of the committee.
Hewitt

its principal

the chair

the committee

#:  3   The principal meets regularly, and as needed, with the committee in executive sessions, has direct access to the chair during and between meetings, and performs no other services for Abbott or its senior executives.
The principal

the committee

executive sessions

direct access

the chair

meetings

no other services

Abbott

its senior e

# create a training examples

In [ ]:
# ("Pizza is a common fast food.", {"entities": [(0, 5, "FOOD")]})
# What if there are more than one entities in one sentence.

In [16]:
nlp_updated = spacy.load(output_dir)
doc=nlp_updated(string)
train_exams=[]
line_num=0

for sent in doc.sents:
    print("#: ", line_num, " ")
    line_num+=1
    entityNumber=0
    entities=[]
    # first, look at noun chunks
    for nouns in sent.noun_chunks:
        if len(nouns)<2: # single nouns
            if nouns.text in CC:
                print('CC token is ', nouns.text)
                token=nouns[0]
                start=token.idx - sent.start_char
                entity=(start, start+len(token),'CC')
                print(entity)
                entities.append(entity)
        elif len(nouns)==2 and nouns.text in CC: # 2 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+1,'CC')
            # length of each word + space
            print(entity)
            entities.append(entity)
        elif len(nouns)==3 and nouns.text in CC: # 3 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+2,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==4 and nouns.text in CC: # 4 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+3,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==5 and nouns.text in CC: # 5 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+4,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==6 and nouns.text in CC: # 6 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+5,'CC')
            print(entity)
            entities.append(entity)   
        elif len(nouns)==7 and nouns.text in CC: # 7 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+6,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==8 and nouns.text in CC: # 8 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+len(nouns[7])+7,'CC')
            print(entity)
            entities.append(entity)
    if entities==[]:
        tpl=(sent.text,{'entities': []})
        train_exams.append(tpl)        
    else:
        tpl=(sent.text,{'entities': entities})
        train_exams.append(tpl)

# deleted entitynumber, 
# try to amek one train example per sentence, not per entity.
# no hit train examples are also produced.


#:  0  
CC chunk token is  F.W. Cook
(0, 9, 'CC')
#:  1  
CC chunk token is  Hewitt Associates
(26, 43, 'CC')
#:  2  
CC token is  Hewitt
(0, 6, 'CC')
#:  3  
#:  4  
CC token is  Hewitt
(57, 63, 'CC')
#:  5  
CC token is  Hewitt
(55, 61, 'CC')
#:  6  
CC token is  Hewitt
(71, 77, 'CC')
#:  7  
#:  8  
CC chunk token is  Aon Hewitt
(26, 36, 'CC')
#:  9  
CC chunk token is  Aon Hewitt
(22, 32, 'CC')
#:  10  
CC token is  Meridian
(91, 99, 'CC')
#:  11  
CC chunk token is  Aon Hewitt
(38, 48, 'CC')
#:  12  
CC token is  Meridian
(22, 30, 'CC')
#:  13  
#:  14  
#:  15  
#:  16  
#:  17  
#:  18  
CC token is  Meridian
(144, 152, 'CC')
CC token is  Meridian
(206, 214, 'CC')
#:  19  
CC token is  Meridian
(22, 30, 'CC')
#:  20  
CC token is  Meridian
(0, 8, 'CC')
#:  21  
CC chunk token is  Mercer Human Resource Consulting
(27, 59, 'CC')
CC token is  Mercer
(61, 67, 'CC')
#:  22  
CC token is  Mercer
(0, 6, 'CC')
#:  23  
CC token is  Mercer
(9, 15, 'CC')
CC chunk token is  Semler Brossy
(

In [17]:
train_exams

[('F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.',
  {'entities': [(0, 9, 'CC')]}),
 ('The committee has engaged Hewitt Associates to provide counsel and advice on executive and non-employee director compensation matters.',
  {'entities': [(26, 43, 'CC')]}),
 ('Hewitt, and its principal, report directly to the chair of the committee.',
  {'entities': [(0, 6, 'CC')]}),
 ('The principal meets regularly, and as needed, with the committee in executive sessions, has direct access to the chair during and between meetings, and performs no other services for Abbott or its senior executives.',
  {'entities': []}),
 ("The committee determines what variables it will instruct Hewitt to consider, and they include: peer groups against which performance and pay should be examined, financial metrics to be used to assess Abbott's relative performance, competitive long-term incentive practi

In [ ]:
#It looks like it worked well.

In [ ]:
# original

In [52]:


nlp_updated = spacy.load(output_dir)
doc=nlp_updated(string)
train_exams=[]
line_num=0

for sent in doc.sents:
    print("#: ", line_num, " ")
    line_num+=1
    entityNumber=0
    # first, look at noun chunks
    for nouns in sent.noun_chunks:
        entities=[]
        if len(nouns)<2: # single nouns
            if nouns.text in CC:
                print('CC token is ', nouns.text)
                token=nouns[0]
                start=token.idx - sent.start_char
                entity=(start, start+len(token),'CC')
                print(entity)
                entities.append(entity)
        elif len(nouns)==2 and nouns.text in CC: # 2 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+1,'CC')
            # length of each word + space
            print(entity)
            entities.append(entity)
        elif len(nouns)==3 and nouns.text in CC: # 3 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+2,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==4 and nouns.text in CC: # 4 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+3,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==5 and nouns.text in CC: # 5 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+4,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==6 and nouns.text in CC: # 6 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+5,'CC')
            print(entity)
            entities.append(entity)   
        elif len(nouns)==7 and nouns.text in CC: # 7 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+6,'CC')
            print(entity)
            entities.append(entity)
        elif len(nouns)==8 and nouns.text in CC: # 8 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+len(nouns[7])+7,'CC')
            print(entity)
            entities.append(entity)
            
        if entities==[]: # if entity was not defined
            continue
        tpl=(sent.text,{'entities': entities})
        train_exams.append(tpl)
        entityNumber+=1
    if entityNumber==0: 
        tpl=(sent.text,{'entities': []})
        train_exams.append(tpl)

#:  0  
CC chunk token is  F.W. Cook
(0, 9, 'CC')
#:  1  
CC chunk token is  Hewitt Associates
(26, 43, 'CC')
#:  2  
CC token is  Hewitt
(0, 6, 'CC')
#:  3  
#:  4  
CC token is  Hewitt
(57, 63, 'CC')
#:  5  
CC token is  Hewitt
(55, 61, 'CC')
#:  6  
CC token is  Hewitt
(71, 77, 'CC')
#:  7  
#:  8  
CC chunk token is  Aon Hewitt
(26, 36, 'CC')
#:  9  
CC chunk token is  Aon Hewitt
(22, 32, 'CC')
#:  10  
CC token is  Meridian
(91, 99, 'CC')
#:  11  
CC chunk token is  Aon Hewitt
(38, 48, 'CC')
#:  12  
CC token is  Meridian
(22, 30, 'CC')
#:  13  
#:  14  
#:  15  
#:  16  
#:  17  
#:  18  
CC token is  Meridian
(144, 152, 'CC')
CC token is  Meridian
(206, 214, 'CC')
#:  19  
CC token is  Meridian
(22, 30, 'CC')
#:  20  
CC token is  Meridian
(0, 8, 'CC')
#:  21  
CC chunk token is  Mercer Human Resource Consulting
(27, 59, 'CC')
CC token is  Mercer
(61, 67, 'CC')
#:  22  
CC token is  Mercer
(0, 6, 'CC')
#:  23  
CC token is  Mercer
(9, 15, 'CC')
CC chunk token is  Semler Brossy
(

In [54]:
train_exams

[('F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.',
  {'entities': [(0, 9, 'CC')]}),
 ('The committee has engaged Hewitt Associates to provide counsel and advice on executive and non-employee director compensation matters.',
  {'entities': [(26, 43, 'CC')]}),
 ('Hewitt, and its principal, report directly to the chair of the committee.',
  {'entities': [(0, 6, 'CC')]}),
 ('The principal meets regularly, and as needed, with the committee in executive sessions, has direct access to the chair during and between meetings, and performs no other services for Abbott or its senior executives.',
  {'entities': []}),
 ("The committee determines what variables it will instruct Hewitt to consider, and they include: peer groups against which performance and pay should be examined, financial metrics to be used to assess Abbott's relative performance, competitive long-term incentive practi

In [55]:
len(train_exams)

113

In [56]:
additionalList=[('F.W. Cook has not provided any other services to the Company and has received no compensation other than with respect to the services provided to the Committee.',
  {'entities': [(0, 9, 'CC')]})
,('The Committee engaged Frederic W. Cook & Co., Inc. (Cook & Co.) in order to advise on compensation matters.',
{'entities': [(22, 50, 'CC')]})
,("Based on its evaluation of Meridian's independence in accordance with the New York Stock Exchange listing standards and information provided by Meridian, the Committee determined that the work performed by Meridian does not present any conflicts of interest.",
  {'entities': [(27,35,'CC'), (144, 152, 'CC'), (206, 214, 'CC')]}),
("With respect to AMD’s 2010 executive compensation program, Compensia’s role was limited to reviewing the amount of each of the Named Executive Officer’s 2010 cash performance bonus and assessing the competitiveness of such bonus payment.",
 {'entities': [(59,68,'CC')]}
 ),('In 2010, neither Semler Brossy nor Compensia provided any services to AMD, or received any payments from AMD, other than in their capacity as a consultant to the Compensation Committee.',
  {'entities': [(17,30,'CC'),(35, 44, 'CC')]})
,('From time to time, the Committee engages CCA Strategies LLC, an employee benefits 6 and compensation consulting firm (which also acts as a consultant to the Human Resources Committee on executive compensation matters), to provide the Committee with information regarding director compensation paid by companies principally in the Fortune 50, Fortune 100 and a special comparator group used by the Human Resources Committee.' ,
 {'entities': [(41,59,'CC')]}
 ),("The Compensation Committee has directly engaged George B. Paulin, Chairman and Chief Executive Officer of Frederic W. Cook & Co., Inc. as its compensation consultant.",
 {'entities': [(106,134,'CC')]} 
 ),("The Compensation Committee has the sole and direct responsibility for the appointment, compensation and oversight of the work of any advisor retained by the Compensation Committee, and it has directly engaged George B. Paulin, Chairman and Chief Executive Officer of Frederic W. Cook & Co., Inc. (Cook & Co.), as its independent compensation consultant."
 , {'entities': [(267,295,'CC'),(297,306,'CC')]}
 ),("He also assisted in the identification of new peer companies for Baxter, in advance of the Baxalta spin-off. In accordance with the rules of the SEC and the NYSE regarding the independence of compensation consultants, Mr. Paulin provided the Compensation Committee information regarding any personal, financial, or business relationships between Cook & Co. and Baxter, its management or the members of the Compensation Committee that could impair its independence or present a conflict of interest."
, {'entities': [(346,355,'CC')]}
),("Based on its review of this information, the Compensation Committee determined that there were no relationships that impair the independence or create a conflict of interest between Baxter and Cook & Co. and the partners, consultants and employees who provide services to the Compensation Committee."
 , {'entities': [(193,202,'CC')]}
),("In addition, the Compensation Committee annually reviews the substantive performance of Mr. Paulin and Cook & Co. as part of its engagement process.",
{'entities': [(103,112,'CC')]}
 ),("A small group of consultants within Mercer Human Resource Consulting (Mercer) has served as the Compensation Committee's executive compensation consultant for a number of years."
 , {'entities': [(36,68,'CC'),(70,76,'CC')]}
 ),("The Committee is aware that other consultants and employees within Mercer provide, and have in the past provided, a range of consulting services to us."
 , {'entities': [(67,73,'CC')]})
,("The consultants from CAP report directly to the Compensation Committee and work with both the Committee (including the Subcommittee) and management to, among other things, provide advice regarding compensation structures in general and competitive compensation data. They also review for the Committee and/or Subcommittee information prepared by management.",
 {'entities': [(21,24,'CC')]})
,("No other services were provided by CAP to the Compensation Committee or to management.",
 {'entities': [(35,38,'CC')]})
,('No other services were provided by Semler Brossy or CAP to the Compensation Committee or to management.',
  {'entities': [(35, 48, 'CC'), (52,55,'CC')]})
,("The Committee utilizes the expertise of an external consultant, Pearl Meyer & Partners, whom the Committee retains and works with during the year and who provides insight into compensation trends and issues.",
 {'entities': [(64, 86, 'CC')]})
,("Pearl Meyer & Partners does not provide other services to ExxonMobil beyond supporting the Committee and assisting the Board Affairs Committee on non-employee director compensation."
 , {'entities': [(0,22, 'CC')]})
,("The Chair of the Compensation Committee negotiates the terms of Pearl Meyer & Partners’ engagement."
 , {'entities': [(64, 86, 'CC')]})
,("The Committee uses an independent consultant, Pearl Meyer & Partners, to provide information on current developments and practices in director compensation.",
 {'entities': [(46, 68, 'CC')]})
,("Pearl Meyer & Partners is the same consultant retained by the Compensation Committee to advise on executive compensation, but performs no other work for ExxonMobil."
 , {'entities': [(0,22, 'CC')]})
,("The Committee considers factors that could affect Pearl Meyer & Partners’ independence, including that the consultant provides no other services for ExxonMobil other than its engagement by the Committee and the Board Affairs Committee as described above. Based on this review, the Committee has determined the consultant’s work for the Committee to be free from conflicts of interest."
 , {'entities': [(50,72, 'CC')]})]

In [57]:
train_exams=train_exams+additionalList

In [58]:
len(train_exams)

135

In [ ]:
# to-do: Check the quality of train examples.

In [ ]:
doc2=nlp_updated('The Company engaged separate consultants, Towers Watson and The Hay Group, to supplement its internal resources and assist with compensation matters in general and with executive compensation as well.')

In [ ]:
for nouns in doc2.noun_chunks:
    print(nouns.text)
    nounList=list(doc2.noun_chunks)

In [ ]:
for elem in nounList:
    if str(elem) in CC:
        print(elem)

In [ ]:
train_exams=[]
line_num=0

for sent in doc2.sents:
    print("#: ", line_num, " ")
    line_num+=1
    # first, look at noun chunks
    for nouns in sent.noun_chunks:
        entities=[]
        if len(nouns)<2: # single nouns
            if nouns.text in CC:
                print('CC token is ', nouns.text)
                token=nouns[0]
                start=token.idx - sent.start_char
                entity=(start, start+len(token),'CC')
                print(entity)
                entities.append(entity)
        elif len(nouns)==2 and nouns.text in CC: # 2 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+1,'CC')
            # length of each word + space
            print(entity)
            entities.append(entity)
            
        elif len(nouns)==3 and nouns.text in CC: # 3 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+2,'CC')
            print(entity)
            entities.append(entity)
            
        elif len(nouns)==4 and nouns.text in CC: # 4 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+3,'CC')
            print(entity)
            entities.append(entity)
                    
        elif len(nouns)==5 and nouns.text in CC: # 5 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+4,'CC')
            print(entity)
            entities.append(entity)
 
        elif len(nouns)==6 and nouns.text in CC: # 6 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+5,'CC')
            print(entity)
            entities.append(entity)
            
        elif len(nouns)==7 and nouns.text in CC: # 7 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+6,'CC')
            print(entity)
            entities.append(entity)
 
        elif len(nouns)==8 and nouns.text in CC: # 8 noun phrases
            print('CC chunk token is ', nouns.text)
            token=nouns[0]
            start=token.idx - sent.start_char
            entity=(start, start+len(nouns[0])+len(nouns[1])+len(nouns[2])+len(nouns[3])+len(nouns[4])+len(nouns[5])+len(nouns[6])+len(nouns[7])+7,'CC')
            print(entity)
            entities.append(entity)
        
        if entities==[]: # if entity was not defined
            continue
        tpl=(sent.text,{'entities': entities})
        train_exams.append(tpl)  
        
    
                


In [ ]:
train_exams

In [ ]:
# To-do
# 0. check why some sentences do not have a hit.
# 1. train
# 2. check the results
# 3. reiterate